In [202]:
import pandas as pd
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

True

In [203]:
# Get paths from environment variables
base_path = os.getenv('HTML_PATH')
download_path = os.getenv('DOWNLOAD_PATH')

# Construct full path to HTML file
# path = os.path.join(base_path, "1.htm")
path = os.path.join(base_path, "DetailedStatement.htm")


In [204]:
# Check if file exists before reading
if not os.path.exists(path):
    print(f"Error: HTML file not found at {path}")
else:
    # Read HTML file
    tables = pd.read_html(path)
    
    # Check if tables are found
    if not tables:
        print("No tables found in the HTML file.")
    else:
        print(f"Found {len(tables)} tables in the HTML file.")
        for i, table in enumerate(tables, 1):
            print(f"Table {i}: {table.shape[0]} rows x {table.shape[1]} columns")

Error: HTML file not found at C:\Users\calli\Downloads\PROP-database-new\da_aggiungere\fnext\100\f\DetailedStatement.htm


In [205]:
def html_to_excel(html_file, excel_file):
    try:
        # Read HTML tables from the file
        tables = pd.read_html(html_file)
        if not tables:
            print("No tables found in the HTML file.")
            return
        
        # Save each table to a separate sheet in Excel
        with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
            for idx, table in enumerate(tables, start=1):
                sheet_name = f"Sheet{idx}"
                table.to_excel(writer, sheet_name=sheet_name, index=False)
        
        print(f"Conversion successful!")
    
    except FileNotFoundError:
        print(f"Error: File '{html_file}' not found.")
    except ValueError as e:
        print(f"Error reading HTML: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

# Convert the HTML file to Excel
if os.path.exists(path):
    excel_output = os.path.join(download_path, "converted_tables.xlsx")
    html_to_excel(path, excel_output)
else:
    print("Cannot convert: HTML file not found.")

Cannot convert: HTML file not found.


In [ ]:
# read converted csv file
df = pd.read_excel(excel_output, sheet_name='Sheet1')
# df.head()

In [ ]:
"""conditions always valid"""

# keep rows from row 2
df = df.iloc[2:].reset_index(drop=True)

# RangeIndex have the corrected column names
df.columns = df.iloc[0]
df = df[1:]

# there are two columns with the same name 'Price'
# rename the second 'Price' column to 'Price2'
df.columns = df.columns.where(df.columns.duplicated() == False, df.columns + '2')

# trim whitespace from column names
df.columns = df.columns.str.strip()

# # trim Profit column values
# df['Profit'] = df['Profit'].str.strip()

# # replace empty characters with '' in 'Profit' column
# df['Profit'] = df['Profit'].replace(r'^\s*$', '', regex=True)

#  drop 'Taxes' column
df = df.drop(columns=['Taxes'], errors='ignore')

In [ ]:
df.tail(20)

In [ ]:
# ==================================================================
"""
from df.tail() check first row with SumSummary:	Summary:	Summary:
cancel all rows from that row onwards, indicating the RangeIndex num
"""
# ==================================================================

'\nfrom df.tail() check first row with SumSummary:\tSummary:\tSummary:\ncancel all rows from that row onwards, indicating the RangeIndex num\n'

In [ ]:
"""conditions depending on data, check data length first"""
# drop rows indicating correct RangeIndex (with iloc)
up_to_row = 1356  # adjust this number based on actual data
df = df.iloc[:up_to_row]

In [ ]:
df.tail(10)

In [ ]:
"""conditions always valid"""

"""
drop rows where 'Open Time' and 'Close Time' have the following values:
Closed P/L:
Open Trades:
Open Time
Working Orders:
No transactions
0.00
"""
# drop strings in 'Open Time' and 'Close Time' columns
strings_to_drop = ['Closed P/L:', 'Open Trades:', 'Open Time', 'Working Orders:', 'No transactions', '0.00']
df = df[~df['Open Time'].isin(strings_to_drop)]
df = df[~df['Close Time'].isin(strings_to_drop)]

"""
drop rows where 'Type' have the following values:
balance
"""
# drop strings in 'Type' column
df = df[~df['Type'].isin(['balance'])]

# drop rows where 'Ticket' is nan
df = df.dropna(subset=['Ticket'])      

In [ ]:
"""
drop rows where 'Commission' contains the following values:
cancelled
Floating P/L:
"from #"
"Portfolio"
"folio"  
"to #"
[tp]
[sl]
"""

# drop rows where Commission column contains any of these strings
strings_to_drop_commission = ['cancelled', 'Floating P/L:', 'from #', 'Portfolio', 'folio', 'to #', '[tp]', '[sl]']

# Use str.contains with regex OR pattern to match any of the strings
pattern = '|'.join(strings_to_drop_commission)
df = df[~df['Commission'].str.contains(pattern, case=False, na=False)]

In [ ]:
# # print df.Commission.unique()
# print(df.Commission.unique())

In [ ]:
# convert Open Time and Close Time to datetime
df['Open Time'] = pd.to_datetime(df['Open Time'])
df['Close Time'] = pd.to_datetime(df['Close Time'])

# drop rows where 'Open Time' is nan
df = df.dropna(subset=['Open Time'])

# replace rows where 'Close Time' is nan with 'Close Time' = 'Open Time' + 4 hours
df['Close Time'] = df['Close Time'].fillna(df['Open Time'] + pd.Timedelta(hours=4))

In [2]:
# df.head()

In [3]:
# df.tail(20)

In [1]:
# # print df.Profit.unique()
# print(df.Profit.unique())

In [ ]:
# clean Profit column before converting to float
# remove spaces used as thousands separators
df['Profit'] = df['Profit'].astype(str).str.replace(' ', '', regex=False)

# convert cleaned dataframe to proper Dtypes
df = df.astype({
    'Size': 'float64',
    # check if 'Ticket' can be converted to int, if not use 'str' instead
    'Ticket': 'int64',
    'S / L': 'float64',
    'T / P': 'float64',
    'Price': 'float64',
    'Price2': 'float64',
    'Commission': 'float64',
    'Swap': 'float64',
    'Profit': 'float64'
})

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 671 entries, 1 to 1344
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Ticket      671 non-null    int64         
 1   Open Time   671 non-null    datetime64[ns]
 2   Type        671 non-null    object        
 3   Size        671 non-null    float64       
 4   Item        671 non-null    object        
 5   Price       671 non-null    float64       
 6   S / L       671 non-null    float64       
 7   T / P       671 non-null    float64       
 8   Close Time  671 non-null    datetime64[ns]
 9   Price2      671 non-null    float64       
 10  Commission  671 non-null    float64       
 11  Swap        671 non-null    float64       
 12  Profit      671 non-null    float64       
dtypes: datetime64[ns](2), float64(8), int64(1), object(2)
memory usage: 73.4+ KB


In [ ]:
# print shape of cleaned dataframe
print(f"Cleaned DataFrame shape: {df.shape}")

# print column names
print("Column names:", df.columns.tolist())

Cleaned DataFrame shape: (671, 13)
Column names: ['Ticket', 'Open Time', 'Type', 'Size', 'Item', 'Price', 'S / L', 'T / P', 'Close Time', 'Price2', 'Commission', 'Swap', 'Profit']


In [ ]:
# ==================================================================
"""
rename cleaned dataframe to clean Excel file name in the next cell
"""
# ==================================================================

In [ ]:
# download cleaned dataframe to excel
cleaned_excel_output = os.path.join(download_path, "cleaned_excel_output.xlsx")

df.to_excel(cleaned_excel_output, index=False)